In [37]:
from dotenv import load_dotenv
import sys, os
import re, random, pandas as pd
from collections import defaultdict

In [47]:

# Load environment variables from .env file
load_dotenv()

# Get the API key
GENIUS_CLIENT_ACCESS_TOKEN = os.getenv("GENIUS_API_KEY1")
SPOTIPY_CLIENT_ID = os.getenv("CLIENT_ID")
SPOTIPY_CLIENT_SECRET = os.getenv("CLIENT_SECRET")


# Use the API key
print(f"Your API key is: {GENIUS_CLIENT_ACCESS_TOKEN}")

Your API key is: Ax0MXEu92RM6-EjFKzU4qWbtJrliPyhP-6vH1VtK8x4USJkR9mMUViZGigfrcjUj


In [59]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import lyricsgenius
from spotipy.oauth2 import SpotifyClientCredentials

# Spotify API credentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
# Initialize the Spotify API client
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=40) #spotify object to access API

# Create a Genius API client
genius = lyricsgenius.Genius(GENIUS_CLIENT_ACCESS_TOKEN, timeout=40)

In [60]:
# import lyricsgenius
# genius = lyricsgenius.Genius(
#     GENIUS_CLIENT_ACCESS_TOKEN,
#     sleep_time=10,       # wait 5 seconds between calls
#     retries=3,          # retry up to 3 times on failure
#     timeout=15,
    
# )


In [61]:
from lyricsgenius.api.api import PublicAPI

public = PublicAPI(
    response_format='plain',
    timeout=20,
    sleep_time=10,   # seconds between calls
    retries=3
)

# Perform a search (this hits `/api/search`)
res = public.search("Die with a smile", per_page=5)
# Find the first song hit
song_hit = next(h for h in res["hits"] if h["type"] == "song")
song_id  = song_hit["result"]["id"]

# Retrieve song metadata (no lyrics text—Genius API doesn’t return lyrics)
song_data = public.song(song_id)
print(song_data)

{'song': {'_type': 'song', 'annotation_count': 2, 'api_path': '/songs/10766684', 'apple_music_id': '1762656732', 'apple_music_player_url': 'https://genius.com/songs/10766684/apple_music_player', 'artist_names': 'Lady Gaga & Bruno Mars', 'comment_count': 157, 'custom_header_image_url': None, 'custom_song_art_image_url': 'https://images.genius.com/abe185baf2b9fd84ebb5d493ffe715b3.1000x1000x1.png', 'description': {'plain': '“Die With A Smile” was the comeback single by Lady Gaga and Bruno Mars, who both had not released music in two to three years. The song was surprise announced a day before the release on August 15, and released at 12am EST alongside the music video.\n\nGaga confirmed on the announcement post that this song is a song to hold fans over until she releases her highly anticipated seventh studio album, which she has teased throughout all of 2024; “Die With A Smile” would later be included as the last track on the album, titled MAYHEM, when it was released in 2025.\n\nGaining

In [62]:
# This function is to get lyrics
def lyrics(song, artist):
    # genius = lyricsgenius.Genius('')
    song = genius.search_song(title=song, artist=artist)
    try:
      lyrics = song.lyrics
      print(lyrics)
      print('\n')
    except AttributeError:
      print("There are no lyrics for this song on Genius")
      print('\n')

In [64]:

song = "Die with a smile"
artist = "Bruno Mars, Lady Gaga"

lyrics(song, artist)


Searching for "Die with a smile" by Bruno Mars, Lady Gaga...
Done.
186 ContributorsTranslationsTürkçeEspañolPortuguêsDeutschFrançaisItalianoالعربيةΕλληνικάTiếng ViệtBahasa IndonesiaفارسیNederlands中文ไทย / Phasa ThaiРусскийPolskiМакедонски日本語HebrewУкраїнськаСрпскиČeskyසිංහලDie With A Smile Lyrics“Die With A Smile” was the comeback single by Lady Gaga and Bruno Mars, who both had not released music in two to three years. The song was surprise announced a day before the release on August 15, and… Read More [Intro: Bruno Mars]
(Ooh, ooh)

[Verse 1: Bruno Mars]
I, I just woke up from a dream
Where you and I had to say goodbye
And I don't know what it all means
But since I survived, I realized

[Pre-Chorus: Bruno Mars]
Wherever you go, that's where I'll follow
Nobody's promised tomorrow
So I'ma love you every night like it's the last night
Like it's the last night

[Chorus: Bruno Mars]
If the world was ending
I'd wanna be next to you
If the party was over
And our time on Earth was through
I'd

In [65]:
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ', '-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ', '-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/' + artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics


In [66]:
def lyrics_onto_frame(df1, artist_name):
    for i, x in enumerate(df1['Track Name']):
        test = scrape_lyrics(artist_name, x)
        print(test)
        df1.loc[i, 'Lyrics'] = test
    return df1

In [75]:
# ------------------------------------------------------------------
# Build the “Student_dataset” – grab ≈400 candidate tracks, then keep 200 random rows
# ------------------------------------------------------------------

target_genre   = "Screamo"   # ← change if you pick another extra-genre
songs_required = 150
batch          = 50        # Spotify’s per-call limit
max_offset     = 1000      # crawl first 1000 results (0‒950)


In [76]:
# Parallel lists → final CSV columns
artist_names, track_names, release_dates, genres, lyrics = [], [], [], [], []

# Keep (artist, track) pairs so we never double-count
seen = set()

# Shuffle offsets so we sample *random* slices of Spotify’s catalogue
offsets = list(range(0, max_offset, batch))
random.shuffle(offsets)

In [77]:
for offset in offsets:
    if len(lyrics) >= songs_required:
        break

    pop_tracks = sp.search(q=f'genre:"{target_genre}"', type='track',
                           limit=batch, offset=offset)

    for track in pop_tracks['tracks']['items']:
        if len(lyrics) >= songs_required:
            break

        key = (track['artists'][0]['name'].lower(), track['name'].lower())
        if key in seen:
            continue                               # skip duplicates
        seen.add(key)

        try:
            # ---------- Genius lyrics ----------
            song = genius.search_song(track['name'], track['artists'][0]['name'])
            if not song:
                continue
            l = song.lyrics
            l = re.sub(r'^.*?\n+',  '', l, count=1)   # drop header
            l = re.sub(r'\s+',      ' ', l).strip()   # single-line text
            if len(l) < 200:                          # too short → skip
                continue

            # ---------- Append to dataset ----------
            lyrics.append(l)
            artist_names.append(track['artists'][0]['name'])
            track_names.append(track['name'])
            release_dates.append(track['album']['release_date'][:4])  # keep YYYY
            genres.append(target_genre)

        except Exception as e:
            print(e)
            continue


Searching for "There's No Penguins In Alaska" by Chiodos...
Done.
Searching for "Scene Three - Stomach Tied In Knots" by Sleeping With Sirens...
Done.
Searching for "Diamonds and Why Men Buy Them" by Pierce The Veil...
Done.
Searching for "GODDESS" by Written by Wolves...
Done.
Searching for "I Will Play My Game Beneath The Spin Light" by Brand New...
Done.
Searching for "Genius" by Written by Wolves...
Done.
Searching for "The Past Should Stay Dead" by Emarosa...
Done.
Searching for "Nightmare (The Devil)" by Fame on Fire...
Done.
Searching for "Poetic Tragedy" by The Used...
Done.
Searching for "Resilience" by Pierce The Veil...
Done.
Searching for "No Way..." by Febuary...
Done.
Searching for "Deadbolt" by Thrice...
Done.
Searching for "Translating the Name" by Saosin...
Done.
Searching for "Accidents" by Alexisonfire...
Done.
Searching for "Walls" by Emery...
Done.
Searching for "Bulletproof Love" by Pierce The Veil...
Done.
Searching for "Skin & Bones" by Silverstein...
Done.
Sear

In [78]:
# ------------------------------------------------------------------
# Build DataFrame, randomly down-sample to exactly 200, write CSV
# ------------------------------------------------------------------
df = pd.DataFrame({
    "artist_name":   artist_names,
    "track_name":    track_names,
    "release_date":  release_dates,
    "genre":         genres,
    "lyrics":        lyrics
}).sample(n=songs_required, random_state=42)


In [79]:
df

,artist_name,track_name,release_date,genre,lyrics
73,"Fear, and Loathing in Las Vegas",Return to Zero,2017,Screamo,"I’m not one of them, no I ain’t I’m not one of..."
18,Saosin,3rd Measurement in C,2003,Screamo,"Taking back, overdone Free and safely Souvenir..."
118,Siamese,Chemistry,2024,Screamo,Maybe this chemistry is playing its tricks ins...
78,Fame on Fire,Spiral (Justice),2024,Screamo,Send me into a spiral! [Verse 1] I did my best...
76,The Early November,I Want To Hear You Sad,2002,Screamo,These eyes They're strongly covered in disguis...
...,...,...,...,...,...
71,From First To Last,Secrets Don't Make Friends,2004,Screamo,This place is a bloodbath And we won't be take...
106,AFI,Paper Airplanes (Makeshift Wings),2003,Screamo,"Their 2006 album, Decemberunderground, brings ..."
14,Emery,Walls,2004,Screamo,"Are you listening? We write a thousand pages, ..."
92,Glassjaw,Cosmopolitan Blood Loss,2002,Screamo,"Many ways, many days, many phases down the lin..."


In [52]:
#Save the df to .csv

In [82]:
df.to_csv("data/Student_dataset.csv", index=False, encoding="utf-8")
print("✅  Student_dataset.csv created with shape:", df.shape)

✅  Student_dataset.csv created with shape: (150, 5)
